# Machine Learning

In this notebook, it is time to play with ML. I'm gonna use some tree based algorithms (xgboost and lgbm), some neural nets, and bayesian optimization to hyperparameter tunning. Hope to get good results.

In [1]:
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import datetime

In [2]:
os.getcwd()

'C:\\Users\\Hugo\\Desktop\\hugo\\DataScience\\Kaggle\\kaggle_credit_risk\\notebooks'

In [3]:
# Importing utils 
os.chdir('C:\\Users\\Hugo\\Desktop\\hugo\\DataScience\\Kaggle\\kaggle_credit_risk\\code')

from utils import *

# Data directory
os.chdir('C:\\Users\\Hugo\\Desktop\\hugo\\DataScience\\Kaggle\\kaggle_credit_risk\\data\\treated_data')

In [4]:
train = pd.read_csv('train_eng.csv')
test = pd.read_csv('test_eng.csv')

In [5]:
train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,EXT_SOURCE_2^3,EXT_SOURCE_2^2 EXT_SOURCE_3,EXT_SOURCE_2^2 DAYS_BIRTH,EXT_SOURCE_2 EXT_SOURCE_3^2,EXT_SOURCE_2 EXT_SOURCE_3 DAYS_BIRTH,EXT_SOURCE_2 DAYS_BIRTH^2,EXT_SOURCE_3^3,EXT_SOURCE_3^2 DAYS_BIRTH,EXT_SOURCE_3 DAYS_BIRTH^2,DAYS_BIRTH^3
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,0.018181,0.009637,-654.152107,0.005108,-346.733022,2.353667e+07,0.002707,-183.785678,1.247560e+07,-8.468590e+11
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,0.240927,0.197797,-6491.237078,0.162388,-5329.192190,1.748916e+08,0.133318,-4375.173647,1.435830e+08,-4.712058e+12
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,0.171798,0.225464,-5885.942404,0.295894,-7724.580288,2.016572e+08,0.388325,-10137.567875,2.646504e+08,-6.908939e+12
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,0.275185,0.216129,-8040.528832,0.169746,-6314.981929,2.349331e+08,0.133318,-4959.747997,1.845150e+08,-6.864416e+12
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,0.033616,0.053210,-2076.117157,0.084225,-3286.224555,1.282190e+08,0.133318,-5201.667828,2.029540e+08,-7.918677e+12


This data was previously treated and engineered (by hand, since automated feature engineering was quite expansive and my computer couldn't handle it). Let's begin the ML process.

## Machine Learning process

In [6]:
# Importing validation process
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold

# Ensemble of trees classifiers
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, BaggingClassifier

# Decision tree just for visualization
from sklearn.tree import DecisionTreeClassifier

# Our validation metric
from sklearn.metrics import roc_auc_score

# Importing lgbm and xgboost
import lightgbm as lgb
import xgboost as xgb

Just like on Fail Fast, I'll use some algorithms with default parameters. Then, I'll use xgboost, lgbm and neural nets. And for the best model, I'll tune hyperparameters with Bayesian Optimization.

In [23]:
# models 
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
ada = AdaBoostClassifier()
bc = BaggingClassifier()
etc = ExtraTreesClassifier()
gbc = GradientBoostingClassifier()

# classifiers dicts
classifiers = {
    'Decision Tree': dt,
    'Random Forest': rf,
    'AdaBoost': ada,
    'Bagging Classifier': bc,
    'Extra Tree Classifier': etc,
    'Gradient Boosting': gbc
}

As a validation method, I'll use Stratified Shuffle Split. It creates n different splits on the data, shuffling the samples. This method is necessary in order to find out if our algorithm is overfitting in the training data. A good algorithm should be able to perform well on data not previously seen in the training process. This is called generalization, and it is the key points of machine learning.

In [7]:
sss = StratifiedShuffleSplit(n_splits = 5)

Cool! Let's make a for loop iteration over classifiers and getting metrics. But first, I'll drop the ID column and target column from the training and testing dataframes.

In [8]:
ID_train = train.SK_ID_CURR.values
ID_test = test.SK_ID_CURR.values

y_train = train.TARGET.values

X_train = train.drop(['SK_ID_CURR', 'TARGET'], axis=1).values
X_test = test.drop(['SK_ID_CURR'], axis=1).values

In [75]:
tic_model = datetime.datetime.now()
for model, clf in classifiers.items():
    
    roc_aucs = []
    
    tic_cv = datetime.datetime.now()
    for train_index, val_index in sss.split(X_train, y_train):
        X_train_, y_train_ = X_train[train_index, :], y_train[train_index]
        X_val_, y_val_ = X_train[val_index, :], y_train[val_index]
        
        clf.fit(X_train_, y_train_)
        roc_aucs.append(roc_auc_score(y_val_, clf.predict_proba(X_val_)[:,1]))
    toc_cv = datetime.datetime.now()
    
    print('Classifier: ' + model)
    print('-------------------')
    print(roc_aucs)
    print('AUC: {} +/- {}'.format(np.array(roc_aucs).mean(), np.array(roc_aucs).std()))
    print("Time elapsed: {} minutes and {} seconds".format(int((toc_cv - tic_cv).seconds / 60), 
                                                           int((toc_cv - tic_cv).seconds % 60)))
    print('='*20)

toc_model = datetime.datetime.now()
print()   
print("Total time elapsed: {} minutes and {} seconds".format(int((toc_model - tic_model).seconds / 60),
                                                             int((toc_model - tic_model).seconds % 60)))

Classifier: Decision Tree
-------------------
[0.539914882234249, 0.539380105635225, 0.5400495700880246, 0.5406251134834923, 0.5475818608028813]
AUC: 0.5415103064487745 +/- 0.0030615101994478666
Time elapsed: 6 minutes and 55 seconds
Classifier: Random Forest
-------------------
[0.6413934611027277, 0.6486663273968813, 0.6504979055497364, 0.649072114803174, 0.6494542883827652]
AUC: 0.6478168194470569 +/- 0.003268837402287209
Time elapsed: 3 minutes and 9 seconds
Classifier: AdaBoost
-------------------
[0.7464020613082756, 0.7574549705694789, 0.7592203730209601, 0.7467749389470386, 0.7537020176123673]
AUC: 0.752710872291624 +/- 0.005308461885649792
Time elapsed: 16 minutes and 19 seconds
Classifier: Bagging Classifier
-------------------
[0.646937282402861, 0.6487526407417195, 0.6475231546214709, 0.6504828468379277, 0.6317010074392173]
AUC: 0.6450793864086393 +/- 0.006798455507822953
Time elapsed: 44 minutes and 5 seconds


C:\Users\Hugo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Classifier: Extra Tree Classifier
-------------------
[0.6542854565027115, 0.6393832584194475, 0.6525781305305951, 0.6507107861563624, 0.6511619064112601]
AUC: 0.6496239076040753 +/- 0.0052702013675929435
Time elapsed: 1 minutes and 46 seconds
Classifier: Gradient Boosting
-------------------
[0.7647890191132665, 0.7649323475618499, 0.7483139620885462, 0.7570576171815314, 0.7612006862270643]
AUC: 0.7592587264344516 +/- 0.006183495292752015
Time elapsed: 50 minutes and 8 seconds

Total time elapsed: 122 minutes and 24 seconds


That's good. GradientBoosting had the best performance. I believe it could be improved by using better algorithms, such as XGBoost and LightGBM.

# LightGBM

LightGBM has became very popular because it implements gradient boosting machines in such a light way compared to XGBoost and others implementations. It is perfect for people with low RAM (such as me). Let's give it a try.

In [9]:
lgb_params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'learning_rate': 0.2 ,
        'verbose': 0,
        'num_leaves': 100,
        'bagging_fraction': 0.95,
        'bagging_freq': 1,
        'bagging_seed': 1,
        'feature_fraction': 0.9,
        'feature_fraction_seed': 1,
        'max_bin': 256,
        'num_rounds': 100,
        'metric' : 'auc'
    }

In [10]:
model = lgb.LGBMClassifier(**lgb_params)

In [14]:
tic_cv = datetime.datetime.now()
roc_aucs = []
for train_index, val_index in sss.split(X_train, y_train):
    X_train_, y_train_ = X_train[train_index, :], y_train[train_index]
    X_val_, y_val_ = X_train[val_index, :], y_train[val_index]

    model.fit(X_train_, y_train_)
    roc_aucs.append(roc_auc_score(y_val_, model.predict_proba(X_val_)[:,1]))
    
toc_cv = datetime.datetime.now()

print('Classifier: LGBM')
print('-------------------')
print(roc_aucs)
print('AUC: {} +/- {}'.format(np.array(roc_aucs).mean(), np.array(roc_aucs).std()))
print("Time elapsed: {} minutes and {} seconds".format(int((toc_cv - tic_cv).seconds / 60), 
                                                       int((toc_cv - tic_cv).seconds % 60)))
print('='*20)

C:\Users\Hugo\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\Hugo\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\Hugo\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\Hugo\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\Hugo\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `nu

Classifier: LGBM
-------------------
[0.7542896207992694, 0.7570891165304523, 0.7509195466310534, 0.75516050442667, 0.7462740123946162]
AUC: 0.7527465601564123 +/- 0.0038025731261801137
Time elapsed: 2 minutes and 11 seconds


It is possible to see good results in a few minutes. I think this is the best algorithm for us to work with. Let's try to optimize its parameters with Bayesian Optimization using the hyperopt library and the Tree Parzen Estimators (TPE) algorithm.

# Bayesian Optimization for LGBM

First, let us define our search space for LGBM. The hyperparameters must be defined as some sort of propabilistic distribution. They are the priors that serves as input to our bayesian optimization algorithm. The hyperopt library offers some default distributions, such as uniform, log uniform and others.

In [25]:
# Define the search space
from hyperopt import hp

space_lgbm = {
    'class_weight': hp.choice('class_weight', [None, 'balanced']),
    'boosting_type': hp.choice('boosting_type', [{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)}, 
                                                 {'boosting_type': 'dart', 'subsample': hp.uniform('dart_subsample', 0.5, 1)},
                                                 {'boosting_type': 'goss', 'subsample': 1.0}]),
    'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0)
}

After defining our search space, in which the Tree Parzen Estimators algorithm is going to make some assumptions about the probabilistic surface it will optimize, let's create a file to follow up the results of the optimization.

In [14]:
from hyperopt import tpe, Trials, STATUS_OK, fmin, space_eval
import csv

# optimization algorithm
tpe_algorithm = tpe.suggest

# Keep track of results
bayes_trials = Trials()

# create the lgbm Dataset
train_set = lgb.Dataset(X_train, label = y_train)

# Output file
out_file = 'gbm_trials.csv'
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)

# Write the headers to the file
writer.writerow(['loss', 'params', 'iteration', 'estimators', 'train_time'])
of_connection.close()

Than, in order to our algorithm to work, it is necessary to create a function it will optimize. I'll be writing specifically for LGBM, but later in this notebook I'll write a more generic function that supports other algorithms. 

This function must return at least 2 parameters: the loss, which is the objective to optimize, and a flag called STATUS_OK, which serves as a go on flag. The other parameters returned are optional, but is interesting to keep track of them.

In [23]:
from timeit import default_timer as timer

N_FOLDS = 5

def objective_lgbm(params, n_folds = N_FOLDS):
    """Objective function for Gradient Boosting Machine Hyperparameter Optimization"""
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    # Retrieve the subsample if present otherwise set to 1.0
    subsample = params['boosting_type'].get('subsample', 1.0)
    
    # Extract the boosting type
    params['boosting_type'] = params['boosting_type']['boosting_type']
    params['subsample'] = subsample
    
    # Make sure parameters that need to be integers are integers
    for parameter_name in ['num_leaves', 'subsample_for_bin', 'min_child_samples']:
        params[parameter_name] = int(params[parameter_name])
    
    start = timer()
    
    # Perform n_folds cross validation
    cv_results = lgb.cv(params, train_set, num_boost_round = 10000, nfold = n_folds, 
                        early_stopping_rounds = 100, metrics = 'auc', seed = 50)
    
    run_time = timer() - start
    
     # Extract the best score
    best_score = np.max(cv_results['auc-mean'])
    
    # Loss must be minimized
    loss = 1 - best_score
    
    # Boosting rounds that returned the highest cv score
    n_estimators = int(np.argmax(cv_results['auc-mean']) + 1)

    # Write to the csv file ('a' means append)
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, params, ITERATION, n_estimators, run_time])
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'iteration': ITERATION,
            'estimators': n_estimators, 
            'train_time': run_time, 'status': STATUS_OK}

After all set up, let's call the fmin function, which will call the objective_lgbm and the search space it will covers to optimize.

This algorithms works as follows:
- Build a surrogate probability model of the objective function
- Find the hyperparameters that perform best on the surrogate
- Apply these hyperparameters to the true objective function
- Update the surrogate model incorporating the new results
- Repeat steps 2–4 until max iterations or time is reached

If you want to dig deeper into this process, read the [excelent article](https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f) on the subject and the [original paper](https://app.sigopt.com/static/pdf/SigOpt_Bayesian_Optimization_Primer.pdf).

In [ ]:
# Global variable
global  ITERATION

ITERATION = 0
MAX_EVALS = 100

# Run optimization
best = fmin(fn = objective_lgbm, space = space_lgbm, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials, rstate = np.random.RandomState(50))

  5%|▌         | 5/100 [19:38<6:45:28, 256.09s/it, best loss: 0.2330311008445396]

C:\Users\Hugo\Anaconda3\lib\site-packages\lightgbm\callback.py:189: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')



In [36]:
space_eval(space_lgbm, best)

{'boosting_type': {'boosting_type': 'goss', 'subsample': 1.0},
 'class_weight': 'balanced',
 'colsample_bytree': 0.7847366868237033,
 'learning_rate': 0.022060926302396058,
 'min_child_samples': 190.0,
 'num_leaves': 105.0,
 'reg_alpha': 0.830079188734248,
 'reg_lambda': 0.8610765653882214,
 'subsample_for_bin': 280000.0}

In [16]:
# Sort the trials with lowest loss (highest AUC) first
bayes_trials_results = sorted(bayes_trials.results, key = lambda x: x['loss'])
bayes_trials_results[:2]

[{'loss': 0.22288786482334866,
  'params': {'boosting_type': 'goss',
   'class_weight': 'balanced',
   'colsample_bytree': 0.7847366868237033,
   'learning_rate': 0.022060926302396058,
   'min_child_samples': 190,
   'num_leaves': 105,
   'reg_alpha': 0.830079188734248,
   'reg_lambda': 0.8610765653882214,
   'subsample_for_bin': 280000,
   'subsample': 1.0},
  'iteration': 41,
  'estimators': 32,
  'train_time': 0.3195206219997999,
  'status': 'ok'},
 {'loss': 0.22427035330261136,
  'params': {'boosting_type': 'dart',
   'class_weight': 'balanced',
   'colsample_bytree': 0.8567164455106517,
   'learning_rate': 0.011726613176605479,
   'min_child_samples': 185,
   'num_leaves': 116,
   'reg_alpha': 0.9133969680917509,
   'reg_lambda': 0.12344041899551565,
   'subsample_for_bin': 280000,
   'subsample': 0.787226858089079},
  'iteration': 68,
  'estimators': 37,
  'train_time': 103.57025506900027,
  'status': 'ok'}]

In [19]:
results = pd.read_csv('gbm_trials.csv')

# Sort with best scores on top and reset index for slicing
results.sort_values('loss', ascending = True, inplace = True)
results.reset_index(inplace = True, drop = True)
results.head()

,loss,params,iteration,estimators,train_time
0,0.222888,"{'boosting_type': 'goss', 'class_weight': 'bal...",41,32,0.319521
1,0.224270,"{'boosting_type': 'dart', 'class_weight': 'bal...",68,37,103.570255
2,0.224654,"{'boosting_type': 'dart', 'class_weight': 'bal...",66,33,98.669069
3,0.225384,"{'boosting_type': 'dart', 'class_weight': 'bal...",71,37,113.597497
4,0.226190,"{'boosting_type': 'dart', 'class_weight': 'bal...",28,196,95.640259


When the results are saved into a csv file, it gets converted into a string. Let's use the ast function to convert it back.

In [20]:
import ast

# Convert from a string to a dictionary
ast.literal_eval(results.loc[0, 'params'])

{'boosting_type': 'goss',
 'class_weight': 'balanced',
 'colsample_bytree': 0.7847366868237033,
 'learning_rate': 0.022060926302396058,
 'min_child_samples': 190,
 'num_leaves': 105,
 'reg_alpha': 0.830079188734248,
 'reg_lambda': 0.8610765653882214,
 'subsample_for_bin': 280000,
 'subsample': 1.0}

# Evaluating best results

Now it's time to evaluate out results. Hope to get good AUC with this tunned model.

In [21]:
# Extract the ideal number of estimators and hyperparameters
best_bayes_estimators = int(results.loc[0, 'estimators'])
best_bayes_params = ast.literal_eval(results.loc[0, 'params']).copy()

# Re-create the best model and train on the training data
best_bayes_model = lgb.LGBMClassifier(n_estimators = best_bayes_estimators, n_jobs = -1, 
                                       objective = 'binary', random_state = 50, **best_bayes_params)
best_bayes_model.fit(X_train, y_train)

LGBMClassifier(boosting_type='goss', class_weight='balanced',
        colsample_bytree=0.7847366868237033, importance_type='split',
        learning_rate=0.022060926302396058, max_depth=-1,
        min_child_samples=190, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=32, n_jobs=-1, num_leaves=105, objective='binary',
        random_state=50, reg_alpha=0.830079188734248,
        reg_lambda=0.8610765653882214, silent=True, subsample=1.0,
        subsample_for_bin=280000, subsample_freq=0)

In [23]:
# Creating submission file

pd.DataFrame(
    {
        'SK_ID_CURR': ID_test,
        'TARGET': best_bayes_model.predict_proba(X_test)[:,1]
    }
).to_csv('submission_lgbm_tunned.csv', index = None)

# Generic Bayesian Optimization Pipeline

I'll be writing a function that accepts scikit-learn models.

In [17]:
def objective(space):
    """Objective function for Generic Model Hyperparameter Optimization"""
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
        
    # KFold cross-validation
    kf = StratifiedKFold(n_splits = 5)
    
    # Getting the model
    model = space['model'](**space['params'])

    # Starting
    start = timer()
    
    # Perform n_folds cross validation
    roc_aucs = []
    for train_index, val_index in kf.split(X_train[:1000,:], y_train[:1000]):
        X_train_, y_train_ = X_train[train_index, :], y_train[train_index]
        X_val_, y_val_ = X_train[val_index, :], y_train[val_index]

        # Change the model here
        model.fit(X_train_, y_train_)
        roc_aucs.append(roc_auc_score(y_val_, model.predict_proba(X_val_)[:,1]))
    
    run_time = timer() - start
    
     # Extract the best score
    best_score = np.mean(roc_aucs)
    
    # Loss must be minimized
    loss = 1 - best_score

    # Write to the csv file ('a' means append)
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, space, ITERATION, run_time])
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': space, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}

def optimize(objective, space, trials, MAX_EVALS = 120, output_file = 'BayesOpt.csv', random_state = 42):
    
    # Output file
    out_file = output_file
    of_connection = open(out_file, 'w')
    writer = csv.writer(of_connection)

    # Write the headers to the file
    writer.writerow(['loss', 'params', 'iteration', 'train_time'])
    of_connection.close()
    
    # Global variable
    global  ITERATION

    ITERATION = 0

    # Run optimization
    best = fmin(fn = objective, space = space, algo = tpe.suggest, 
                max_evals = MAX_EVALS, trials = trials, rstate = np.random.RandomState(42))
    
    return space_eval(space, best)
    

In [18]:
# Importing utils 
os.chdir('C:\\Users\\Hugo\\Desktop\\hugo\\DataScience\\Kaggle\\kaggle_credit_risk\\spaces')

from search_spaces import *

os.chdir('C:\\Users\\Hugo\\Desktop\\hugo\\DataScience\\Kaggle\\kaggle_credit_risk\\trials')

In [19]:
gbc_space

{'model': sklearn.ensemble.gradient_boosting.GradientBoostingClassifier,
 'params': {'learning_rate': <hyperopt.pyll.base.Apply at 0x1c609c04d68>,
  'subsample': <hyperopt.pyll.base.Apply at 0x1c609c04eb8>,
  'max_depth': <hyperopt.pyll.base.Apply at 0x1c609c0a080>,
  'max_features': <hyperopt.pyll.base.Apply at 0x1c609c0a198>,
  'min_samples_leaf': <hyperopt.pyll.base.Apply at 0x1c609c0a400>,
  'min_samples_split': <hyperopt.pyll.base.Apply at 0x1c609c0a588>}}

In [22]:
rf_trials = Trials()

best_rf = optimize(objective, lgm_space, rf_trials)

  0%|          | 0/120 [00:00<?, ?it/s, best loss: ?]


TypeError: Unknown type of parameter:boosting_type, got:dict